In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords


ModuleNotFoundError: No module named 'tensorflow'

In [25]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [26]:
df.shape

(20800, 5)

In [27]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [28]:
df=df.dropna()

In [29]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [30]:
## Get the Dependent features
y=df['label']

In [32]:
y.shape

(18285,)

In [33]:
### Vocabulary size
voc_size=5000

In [34]:
messages=X.copy()

In [38]:
messages.reset_index(inplace=True)

In [39]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [40]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [42]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [43]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4783, 4761, 1512, 3600, 3630, 1936, 331, 3578, 1873, 1437],
 [1371, 2988, 4569, 6, 2045, 3570, 2554],
 [1235, 3646, 164, 1513],
 [247, 743, 1882, 4796, 3386, 320],
 [4468, 2045, 4117, 1403, 228, 2949, 2045, 3452, 198, 3230],
 [60,
  4568,
  2975,
  1714,
  4847,
  4364,
  4609,
  2849,
  584,
  2891,
  2491,
  2597,
  677,
  3858,
  2554],
 [2757, 210, 4638, 4218, 4489, 4048, 1964, 4307, 3935, 1738, 1952],
 [4927, 576, 2266, 4691, 2792, 1162, 4364, 1509, 3935, 1738, 1952],
 [2221, 3327, 3592, 112, 1589, 3229, 4197, 1367, 4364, 3215],
 [2135, 1177, 1507, 4060, 3512, 2945, 866, 4358],
 [2047, 4412, 1080, 2975, 4028, 1116, 613, 822, 183, 1865, 287],
 [4796, 1778, 3630, 3229, 4364, 2792],
 [3534, 4755, 2617, 4839, 730, 3489, 4212, 2343, 1614],
 [3597, 4496, 1876, 796, 3068, 2270, 316, 3935, 1738, 1952],
 [2002, 4988, 3978, 1513, 1185, 3935, 1738, 1952],
 [1881, 2348, 3530, 2314, 253, 2971, 4125, 437, 1379, 17],
 [4511, 863, 2988],
 [954, 3508, 594, 1738, 4364, 381, 1001, 2554],
 [307, 42

In [44]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [45]:
onehot_repr[1]

[1371, 2988, 4569, 6, 2045, 3570, 2554]

In [46]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4783 4761 1512 ...    0    0    0]
 [1371 2988 4569 ...    0    0    0]
 [1235 3646  164 ...    0    0    0]
 ...
 [2109 2031 2907 ...    0    0    0]
 [2331 2792 2621 ...    0    0    0]
 [2497  965 3168 ...    0    0    0]]


In [47]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
len(embedded_docs),y.shape

(18285, (18285,))

In [49]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [50]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [60]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
192/192 [==============================] - 4s 13ms/step - loss: 0.3360 - accuracy: 0.8386 - val_loss: 0.2070 - val_accuracy: 0.9094
Epoch 2/15
192/192 [==============================] - 2s 11ms/step - loss: 0.1590 - accuracy: 0.9368 - val_loss: 0.2002 - val_accuracy: 0.9180
Epoch 3/15
192/192 [==============================] - 2s 10ms/step - loss: 0.1189 - accuracy: 0.9546 - val_loss: 0.2227 - val_accuracy: 0.9171
Epoch 4/15
192/192 [==============================] - 2s 11ms/step - loss: 0.0946 - accuracy: 0.9664 - val_loss: 0.2466 - val_accuracy: 0.9143
Epoch 5/15
192/192 [==============================] - 2s 11ms/step - loss: 0.0708 - accuracy: 0.9753 - val_loss: 0.2794 - val_accuracy: 0.9140
Epoch 6/15
192/192 [==============================] - 2s 11ms/step - loss: 0.0563 - accuracy: 0.9820 - val_loss: 0.3224 - val_accuracy: 0.9110
Epoch 7/15
192/192 [==============================] - 2s 11ms/step - loss: 0.0446 - accuracy: 0.9849 - val_loss: 0.3588 - val_accuracy: 0.9062

In [53]:
''''
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
''''''

In [61]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [62]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [63]:
from sklearn.metrics import confusion_matrix

In [64]:
confusion_matrix(y_test,y_pred)

array([[3109,  310],
       [ 248, 2368]], dtype=int64)

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9075393537696769

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

